In [8]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

# Dataset

In [3]:
animals = {1:'butterfly', 2:'cat', 3:'cow', 4:'dog', 5:'elephant',6:'hen', 7:'horse', 8:'sheep', 9:'spider', 10:'squirrel'}

In [19]:
path_to_main_dir = '/home/arjun/Desktop/Datasets/animals'

for sub in os.listdir(path_to_main_dir):
    for img in path_to_main_dir + '/'+sub:
        print(os.path.join(path_to_main_dir + ,img))
        break


/
/
/
/
/
/
/
/
/
/
